In [1]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatibl

In [2]:
# %% Method 2

import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

# --- 1. Load DataFrames ---
df_orders = pd.read_csv('orders_day1.csv')
df_distance = pd.read_csv('distance_day1.csv', index_col=0)
df_time = pd.read_csv('time_day1.csv', index_col=0)

# --- PARAMETERS BASED ON MIP FORMULATION ---
# Alpha (α): Coefficient to penalize the number of vehicles. Must be large to prioritize
# vehicle count minimization over total travel time minimization.
ALPHA_VEHICLE_PENALTY = 200

# Physical Fleet Specifications (for sanity checks and fleet sizing)
PHYSICAL_FLEET_SPECS = {
    'weight': 2000,  # Max weight capacity of each truck
    'volume': 1000   # Max volume capacity of each truck
}

# Buffer time to ensure depot is open when vehicles return
BUFFER_RETURN_TIME = 200

# Extra vehicles to add beyond theoretical minimum
ROUTING_BUFFER = 5

# ----------------------------------------------------------------------
# --- 2. Data Structure Preparation for OR-Tools ---
# ----------------------------------------------------------------------

import math

def create_data_model_smart(orders_df, distance_df, time_df, physical_fleet_specs={}):
    """
    Initializes VRP data dynamically based on input statistics to ensure feasibility.

    physical_fleet_specs: dict containing 'weight_limit', 'volume_limit' of your REAL trucks.
    """
    data = {}

    # --- 1. Data Cleaning (Keep existing logic) ---
    distance_df.dropna(how='all', axis=0, inplace=True)
    distance_df.dropna(how='all', axis=1, inplace=True)
    time_df.dropna(how='all', axis=0, inplace=True)
    time_df.dropna(how='all', axis=1, inplace=True)

    data['distance_matrix'] = distance_df.values.astype(int).tolist()
    data['time_matrix'] = time_df.values.astype(int).tolist()

    # --- 2. Smart Time Window Parsing ---
    # Parse actual windows first to find the "Horizon" (Latest possible deadline)
    parsed_windows = []
    max_deadline_in_data = 0

    # Depot is always (0,0) or (0, Open_Duration)
    # Let's assume Depot is open as long as the latest customer needs.
    parsed_windows.append((0, 0))

    for tw_str in orders_df['TIME WINDOW']:
        # Parse the string "(900, 1200)" -> 900, 1200
        clean_str = tw_str.strip('()')
        if ',' in clean_str:
            parts = clean_str.split(',')
            start = int(parts[0])
            end = int(parts[1])
            parsed_windows.append((start, end))
            # Track the latest time anyone needs service
            if end > max_deadline_in_data:
                max_deadline_in_data = end
        else:
            # Fallback for bad data
            parsed_windows.append((0, 10000)) # Default fallback

    # Update Depot's window to extend to the latest deadline (plus return trip buffer)
    # This prevents the "Depot closed before driver returns" error.
    # approx time to drive back to depot after last delivery
    horizon = max_deadline_in_data + BUFFER_RETURN_TIME
    parsed_windows[0] = (0, horizon)

    data['time_windows'] = parsed_windows
    data['vehicle_max_travel_time'] = horizon # Set Horizon dynamically

    # --- 3. Demand & Capacity Sanity Check ---
    data['weights'] = [0] + orders_df['WEIGHT'].round().astype(int).tolist()

    # Notice: Multiplier is consistent (was 100 in your snippet, 1000 in previous. Check your data!)
    data['volumes'] = [0] + (orders_df['VOLUME'] * 100).round().astype(int).tolist()

    # CONSTANTS (Physical limits of your trucks)
    TRUCK_W_CAP = physical_fleet_specs.get('weight', 2000)
    TRUCK_V_CAP = physical_fleet_specs.get('volume', 1000)

    # Sanity Check: Does the biggest order fit in a truck?
    max_order_w = max(data['weights'])
    if max_order_w > TRUCK_W_CAP:
        raise ValueError(f"CRITICAL ERROR: Order exists with weight {max_order_w}, but truck limit is {TRUCK_W_CAP}.")

    # --- 4. Smart Fleet Sizing (The Lower Bound Calculation) ---
    total_weight = sum(data['weights'])
    total_volume = sum(data['volumes'])

    # Minimum trucks needed purely for capacity (Bin Packing Lower Bound)
    min_trucks_weight = math.ceil(total_weight / TRUCK_W_CAP)
    min_trucks_volume = math.ceil(total_volume / TRUCK_V_CAP)

    theoretical_min_vehicles = max(min_trucks_weight, min_trucks_volume)

    # Add a "Routing Buffer" (e.g., 20% or +2 trucks)
    # Vehicles can rarely be 100% full because they run out of Time or Distance first.
    recommended_fleet_size = int(theoretical_min_vehicles * 1.2) + ROUTING_BUFFER

    print(f"--- Initialization Report ---")
    print(f"Total Weight: {total_weight} | Max Truck W: {TRUCK_W_CAP} -> Min Trucks: {min_trucks_weight}")
    print(f"Total Volume: {total_volume} | Max Truck V: {TRUCK_V_CAP} -> Min Trucks: {min_trucks_volume}")
    print(f"Latest Deadline found: {max_deadline_in_data}")
    print(f"Setting Fleet Size to: {recommended_fleet_size} (Theoretical Min: {theoretical_min_vehicles})")

    data['num_vehicles'] = recommended_fleet_size
    data['vehicle_capacities_weight'] = [TRUCK_W_CAP] * data['num_vehicles']
    data['vehicle_capacities_volume'] = [TRUCK_V_CAP] * data['num_vehicles']

    # --- 5. Other Data ---
    data['service_times'] = [0] + orders_df['SERVICE_TIME'].astype(int).tolist()
    data['depot'] = 0
    data['penalty'] = 100000 # Keep high

    return data

# ----------------------------------------------------------------------
# --- 3. Initialize Solver Model and Constraints ---
# ----------------------------------------------------------------------

def print_solution(data, manager, routing, solution):
   """Prints the solution found by the solver."""
   total_distance = 0
   total_time_cost = 0 # Cost is now based on time, not distance
   total_time = 0
   time_dimension = routing.GetDimensionOrDie('Time')

   # Calculate the total travel time for the objective reporting
   for vehicle_id in range(data['num_vehicles']):
       index = routing.Start(vehicle_id)
       if routing.IsEnd(solution.Value(routing.NextVar(index))):
           continue

       while not routing.IsEnd(index):
           previous_index = index
           index = solution.Value(routing.NextVar(index))

           # The actual total travel time component of the objective
           from_node = manager.IndexToNode(previous_index)
           to_node = manager.IndexToNode(index)
           # The cost being minimized is based on the time matrix
           total_time_cost += data['time_matrix'][from_node][to_node]

   # Calculate objective based on MIP formulation: alpha * Yk + sum(tij * xijk)
   num_used_vehicles = len([v for v in range(data['num_vehicles']) if not routing.IsEnd(solution.Value(routing.NextVar(routing.Start(v))))])
   vehicle_penalty_component = num_used_vehicles * ALPHA_VEHICLE_PENALTY

   # NOTE: The OR-Tools objective value here may include dropped node penalties,
   # but the custom calculation below reflects the MIP goal:
   mip_objective = vehicle_penalty_component + total_time_cost

   print(f'OR-Tools Objective (Time Cost + Penalties): {solution.ObjectiveValue()}')
   print(f'MIP Objective (Alpha*Vehicles + Total Time): {mip_objective} ({num_used_vehicles} vehicles * {ALPHA_VEHICLE_PENALTY} + {total_time_cost})')

   # Print routes (unchanged)
   for vehicle_id in range(data['num_vehicles']):
       index = routing.Start(vehicle_id)
       if routing.IsEnd(solution.Value(routing.NextVar(index))):
           continue

       plan_output = f'Route for vehicle {vehicle_id} (Depot: 0):'
       route_distance = 0

       while not routing.IsEnd(index):
           time_var = time_dimension.CumulVar(index)
           node_index = manager.IndexToNode(index)

           previous_index = index
           index = solution.Value(routing.NextVar(index))

           route_distance += routing.GetArcCostForVehicle(
               previous_index, index, vehicle_id
           )

           plan_output += (
               f' {node_index} -> Time({solution.Min(time_var)})'
           )

       time_var = time_dimension.CumulVar(index)
       plan_output += (
           f' {manager.IndexToNode(index)} -> Time({solution.Min(time_var)})'
       )
       plan_output += f'\n  Route Distance: {route_distance}'
       plan_output += f'\n  Route End Time: {solution.Min(time_var)}'

       print(plan_output)
       total_distance += route_distance
       total_time = max(total_time, solution.Min(time_var))

   print(f'\nTotal distance of all used routes: {total_distance}')
   print(f'Max route end time: {total_time}')
   print(f'Total travel time cost: {total_time_cost} (The minimized sum component)')

   # --- Report Dropped Nodes ---
   dropped_nodes = []
#    for node in range(1, len(data['distance_matrix'])):
#        if solution.Value(routing.NextVar(manager.NodeToIndex(node))) == manager.NodeToIndex(node):
#            dropped_nodes.append(node)

   if dropped_nodes:
       print(f'\n!!! WARNING: {len(dropped_nodes)} Nodes Were DROPPED (Unserved) !!!')
       print(f'Dropped Nodes (NODE_ID): {dropped_nodes}')
   else:
       print('\nSUCCESS: All nodes were served.')


def solve_vrp():
   """Entry point for the VRP solver with MIP objective."""
   data = create_data_model_smart(df_orders, df_distance, df_time)

   manager = pywrapcp.RoutingIndexManager(
       len(data['distance_matrix']), data['num_vehicles'], data['depot']
   )

   routing = pywrapcp.RoutingModel(manager)

   # --- A. Define Cost (Time - as per MIP Objective) ---
   def time_cost_callback(from_index, to_index):
       from_node = manager.IndexToNode(from_index)
       to_node = manager.IndexToNode(to_index)
       # The cost minimized is the travel time (t_ij)
       return data['time_matrix'][from_node][to_node]

   transit_callback_index = routing.RegisterTransitCallback(time_cost_callback)
   routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

   # A.1. Set Objective to Minimize Time Cost (t_ij) AND Penalize Vehicles (alpha * y_k)

   # Fixed cost (alpha * y_k): Apply a large penalty for using each vehicle
   for vehicle_id in range(data['num_vehicles']):
       routing.SetFixedCostOfVehicle(ALPHA_VEHICLE_PENALTY, vehicle_id)

   # A.2. Add Dropped Node Penalty
   for node in range(1, len(data['distance_matrix'])):
       routing.AddDisjunction([manager.NodeToIndex(node)], data['penalty'])

   # --- B. Add Capacity Constraints (Weight and Volume) ---

   def add_capacity_dimension(capacity_name, capacities, demands):
       def demand_callback(index):
           node = manager.IndexToNode(index)
           return demands[node]

       demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

       # Capacity bounds are imposed at every node [cite: 87]
       routing.AddDimensionWithVehicleCapacity(
           demand_callback_index,
           0, # Slack
           capacities,
           True, # This dimension is cumulative
           capacity_name
       )

   add_capacity_dimension('WeightCapacity', data['vehicle_capacities_weight'], data['weights'])
   add_capacity_dimension('VolumeCapacity', data['vehicle_capacities_volume'], data['volumes'])


   # --- C. Add Time Window Constraints ---

   def time_callback(from_index, to_index):
       from_node = manager.IndexToNode(from_index)
       to_node = manager.IndexToNode(to_index)
       travel_time = data['time_matrix'][from_node][to_node]
       service_time = data['service_times'][from_node]
       # Time propagation: T_jk >= T_ik + s_i + t_ij [cite: 99]
       return travel_time + service_time

   time_callback_index = routing.RegisterTransitCallback(time_callback)

   # Time dimension: tracks T_ik (time vehicle k starts service at node i) [cite: 45]
   routing.AddDimension(
       time_callback_index,
       0,                             # slack_max (0 is fine here)
       data['vehicle_max_travel_time'], # Planning horizon H [cite: 28]
       False,
       'Time'
   )
   time_dimension = routing.GetDimensionOrDie('Time')

   # Apply Time Windows [cite: 94]
   for node in range(len(data['time_windows'])):
       index = manager.NodeToIndex(node)
       start, end = data['time_windows'][node]
       time_dimension.CumulVar(index).SetRange(start, end)

   # Apply Route Duration Constraint (limited by H) [cite: 104]
   for i in range(data['num_vehicles']):
       time_dimension.SetSpanUpperBoundForVehicle(
           data['vehicle_max_travel_time'],
           i
       )

   # --- D. Set Search Parameters and Solve ---
   search_parameters = pywrapcp.DefaultRoutingSearchParameters()
   search_parameters.first_solution_strategy = (
       routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
   )
   search_parameters.local_search_metaheuristic = (
       routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
   )
   search_parameters.time_limit.seconds = 120

   # Solve the problem
   solution = routing.SolveWithParameters(search_parameters)

   # --- E. Print Solution ---
   if solution:
       print_solution(data, manager, routing, solution)
   else:
       print('No solution found!')


# ----------------------------------------------------------------------
# --- EXECUTION ---
# ----------------------------------------------------------------------

print("Running Vehicle Routing Problem Solver (MIP Objective: Minimize Vehicles, then Time)...")
solve_vrp()

FileNotFoundError: [Errno 2] No such file or directory: 'orders_day1.csv'